In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import rioxarray
from shapely.geometry import mapping
import geopandas as gpd
import xarray as xr
# import numpy as np
# import matplotlib.pyplot as plt
# import geopandas as gpd
# from rasterstats import zonal_stats
# import rasterio
# from rasterio.enums import Resampling
# import matplotlib
# import matplotlib.colors as mcolors
# import xarray as xr
# import cftime
# import math
# import rioxarray
# from shapely.geometry import mapping
# import cartopy.crs as ccrs
# import matplotlib as mpl
# import datetime
# from datetime import timedelta
# import re
# import seaborn as sns
# import calendar
# from mlxtend.evaluate import confusion_matrix
# from mlxtend.plotting import plot_confusion_matrix

#### Set config values

In [3]:
from pathlib import Path
import sys
import os

path_mod = f"{Path(os.path.dirname(os.path.abspath(''))).parents[0]}/"
# print(path_mod)
sys.path.append(path_mod)
from src.indicators.drought.config import Config
from src.utils_general.raster_manipulation import invert_latlon, change_longitude_range

In [4]:
country="malawi"
config=Config()
parameters = config.parameters(country)
country_dir = os.path.join(config.DIR_PATH, config.ANALYSES_DIR, country)
country_data_raw_dir = os.path.join(config.DATA_DIR,config.RAW_DIR,country)
country_data_processed_dir = os.path.join(config.DATA_DIR,config.PROCESSED_DIR,country)
country_data_exploration_dir = os.path.join(config.DATA_DIR,"exploration",country)
dry_spells_processed_dir=os.path.join(country_data_processed_dir,"dry_spells")
chirpsgefs_processed_dir = os.path.join(dry_spells_processed_dir,"chirpsgefs")

#we have different methodologies of computing dryspells and rainy season
#this notebook chooses one, which is indicated by the files being used
chirpsgefs_stats_path=os.path.join(chirpsgefs_processed_dir,"mwi_chirpsgefs_rainyseas_stats_mean_back.csv")
chirps_rolling_sum_path=os.path.join(dry_spells_processed_dir,"data_mean_values_long.csv")

adm1_bound_path=os.path.join(country_data_raw_dir,config.SHAPEFILE_DIR,parameters["path_admin1_shp"])
adm2_bound_path=os.path.join(country_data_raw_dir,config.SHAPEFILE_DIR,parameters["path_admin2_shp"])

In [5]:
def fix_calendar(ds, timevar='F'):
    """
    Some datasets come with a wrong calendar attribute that isn't recognized by xarray
    So map this attribute that can be read by xarray
    Args:
        ds (xarray dataset): dataset of interest
        timevar (str): variable that contains the time in ds

    Returns:
        ds (xarray dataset): modified dataset
    """
    if "calendar" in ds[timevar].attrs.keys():
        if ds[timevar].attrs['calendar'] == '360':
            ds[timevar].attrs['calendar'] = '360_day'
    elif "units" in ds[timevar].attrs.keys():
        if "months since" in ds[timevar].attrs['units']:
            ds[timevar].attrs['calendar'] = '360_day'
    return ds


In [6]:
arc2_url="https://iridl.ldeo.columbia.edu/SOURCES/.NOAA/.NCEP/.CPC/.FEWS/.Africa/.DAILY/.ARC2/.daily/dods"

In [7]:
remote_data = xr.open_dataset(arc2_url,
                              decode_times=False)

In [8]:
remote_data

<xarray.Dataset>
Dimensions:   (T: 13969, X: 751, Y: 801)
Coordinates:
  * T         (T) float32 8.401e+03 8.402e+03 8.403e+03 ... 2.237e+04 2.237e+04
  * Y         (Y) float32 -40.0 -39.9 -39.8 -39.7 -39.6 ... 39.7 39.8 39.9 40.0
  * X         (X) float32 -20.0 -19.9 -19.8 -19.7 -19.6 ... 54.7 54.8 54.9 55.0
Data variables:
    est_prcp  (T, Y, X) float32 ...

In [9]:
remote_data.T.attrs["units"]='days since 1960-01-01 00:00:00 UTC'
remote_data.T.attrs['calendar'] = 'julian'#'360_day' #"julian"#

In [10]:
remote_data=remote_data.rename({"X":"lon","Y":"lat"})
remote_data=fix_calendar(remote_data,timevar="T")
remote_data = xr.decode_cf(remote_data)

In [11]:
remote_data

<xarray.Dataset>
Dimensions:   (T: 13969, lat: 801, lon: 751)
Coordinates:
  * T         (T) object 1983-01-01 00:00:00 ... 2021-03-30 00:00:00
  * lat       (lat) float32 -40.0 -39.9 -39.8 -39.7 ... 39.7 39.8 39.9 40.0
  * lon       (lon) float32 -20.0 -19.9 -19.8 -19.7 ... 54.7 54.8 54.9 55.0
Data variables:
    est_prcp  (T, lat, lon) float32 ...

In [12]:
df_bound=gpd.read_file(adm1_bound_path)

In [ ]:
ds=remote_data.sel(T="2021-03-28").rio.set_spatial_dims(x_dim=config.LONGITUDE, y_dim=config.LATITUDE).rio.write_crs("EPSG:4326").rio.clip(df_bound.geometry.apply(mapping), df_bound.crs, all_touched=True)

In [ ]:
xr.show_versions()